In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split

from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

import lightgbm as lgb


In [ ]:
data = pd.read_csv('../input/tabular-playground-series-jun-2022/data.csv',index_col='row_id')

In [ ]:
data_train = data.copy()

In [ ]:
submission=pd.DataFrame(columns=['row-col','value'])

round_num = 5

for round_i in range(round_num):
    for i in range(len(data_train.columns)):
        data_pred_index=data[data.iloc[:,i].isnull()].index
        data_pred = np.delete(data_train[data.iloc[:,i].isnull()].values,i,axis=1)
        if len(data_pred)>0:
            X_train = np.delete(data_train[data.iloc[:,i].notnull()].values,i,axis=1)
            y_train = data_train[data.iloc[:,i].notnull()].iloc[:,i].values

            print(f'--------------{data_train.columns[i]}---------------')

            lgb_train = lgb.Dataset(X_train, y_train)

            params = {'task': 'train',                
                      'boosting_type': 'gbdt',        
                      'objective': 'regression',          
                      'metric': 'root_mean_squared_error',      #
                      'learning_rate': 0.1,
                      'num_leaves': 220,
                      'min_data_in_leaf': 100,
#                       'device':'gpu'
                     }
            callbacks=[
                          lgb.early_stopping(100,verbose=True),
                          lgb.log_evaluation(-1)
                      ]
            evaluation_results = {}                                    
            model_lgb = lgb.train(params,                                   
                              lgb_train,                              
                              num_boost_round=1000,                    
                              valid_names=['train',],           
                              valid_sets=[lgb_train,],         
                              evals_result=evaluation_results,         
                              callbacks=callbacks,
                              )

            pred_pred = model_lgb.predict(data_pred)
            df_pred_pred = pd.concat([pd.Series(data_pred_index,name='row-col').astype('str')+'-'+data_train.columns[i] , pd.Series(pred_pred,name='value')],axis=1)
            data_train.loc[data.iloc[:,i].isnull(),data.columns[i]]=pd.Series(pred_pred,name='value',index=data_pred_index)
            if round_i == round_num-1:
                submission = pd.concat([submission,df_pred_pred])

In [ ]:
submission

In [ ]:
submission.to_csv('../working/submission.csv',index=False)